In [1]:
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from sklearn.metrics import confusion_matrix
import multiprocessing
import copy
from tqdm import tqdm
import time
from math import sqrt
from skimage import io 
from PIL import Image, ImageFilter
import random
from keras.preprocessing.image import ImageDataGenerator 
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

%matplotlib inline

## Model Testing



In this section we will be testing the CNN as defined in section 4 under two different modifications to its input. Firstly, we will investigate the effect of the image pre-processing in section 3 by training an initial CNN (model) on the unprocessed data, and an alternative CNN (new_model) on the new, augmented data. 

Secondly, we will attempt to test our hypothesis as proposed in section 4: that the angle that the image is taken from is an important factor in the performance of the model. To test this, we will attempt to classify the test set into three categories: 'Side-On', 'Back', and 'Top-Down'. Then we will run the CNN on each category of data to see if there is any noticable difference in prediction accuracy. 

We begin by loading in both the original and preprocessed training data, and the original test data.

*Note: We will be loading in a lot of data and training multiple neural networks in this section, so to conserve memory we now resize the images to 100x100.*

In [2]:
# LOADING THE DATA

#Defining our four class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

#No. of classes
nb_classes = len(class_names)

#Image pixel width x height
IMAGE_SIZE = (100, 100)

#Loading Training and Testing folders
datasets = ['/kaggle/input/brain-tumor-mri-dataset/Training','/kaggle/input/processed-brain-data/Training','/kaggle/input/brain-tumor-mri-dataset/Testing']
output = []
#Perform for both Training and Testing
for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        #Searching through each folder corresponding to a class
        for folder in class_names:
            label = class_names_label[folder]
            
            #Searching through every image within the folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Getting path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Opening and resizing image
                image = cv2.imread(img_path)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its label to the output
                images.append(image)
                labels.append(label)
        
        #Convert to array
        images = np.array(images)
        labels = np.array(labels)   
        
        output.append((images, labels))

#Define training and testing images and labels
#(train_images, train_labels) = output
#train_images = output[0][0]
#train_labels = output[0][1]
(train_images, train_labels), (new_train_images, new_train_labels), (test_images, test_labels) = output

Loading /kaggle/input/brain-tumor-mri-dataset/Training


100%|██████████| 1457/1457 [00:10<00:00, 136.46it/s]


Loading /kaggle/input/processed-brain-data/Training


100%|██████████| 5893/5893 [00:51<00:00, 114.98it/s]


Loading /kaggle/input/brain-tumor-mri-dataset/Testing


100%|██████████| 300/300 [00:02<00:00, 149.32it/s]


Then to label the angles of the testing data, we use the images outputted by section 6

In [3]:
# LOADING THE DATA

#Defining our four class names
class_names = ['back', 'sideon', 'topdown']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

#No. of classes
nb_classes = len(class_names)

#Image pixel width x height
IMAGE_SIZE = (100, 100)

#Loading Training and Testing folders
datasets = ['/kaggle/input/shreyas-weird-images/Train and Output Combined']
output = []
#Perform for both Training and Testing
for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        #Searching through each folder corresponding to a class
        for folder in class_names:
            label = class_names_label[folder]
            
            #Searching through every image within the folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Getting path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Opening and resizing image
                image = cv2.imread(img_path)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its label to the output
                images.append(image)
                labels.append(label)
        
        #Convert to array
        images = np.array(images)
        labels = np.array(labels)   
        
        output.append((images, labels))

#Define training and testing images and labels
#(train_images, train_labels) = output
angle_train_images = output[0][0]
angle_train_labels = output[0][1]

Loading /kaggle/input/shreyas-weird-images/Train and Output Combined


100%|██████████| 9041/9041 [01:22<00:00, 109.12it/s]


We begin by defining both our CNNs in exactly the same way, as in section 4, and then train one off the initial data and one off the preproccesed data.

In [4]:
#BUILDING INITIAL MODEL
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

#Compiling model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
#TRAINING CNN
model.fit(train_images, train_labels, batch_size=55, epochs=20, validation_split = 0.2)

Epoch 1/20
84/84 [==============================] - 8s 12ms/step - loss: 6.6521 - accuracy: 0.6518 - val_loss: 1.2344 - val_accuracy: 0.5171
Epoch 2/20
84/84 [==============================] - 1s 9ms/step - loss: 0.3045 - accuracy: 0.8928 - val_loss: 0.9069 - val_accuracy: 0.7174
Epoch 3/20
84/84 [==============================] - 1s 9ms/step - loss: 0.1732 - accuracy: 0.9396 - val_loss: 0.8006 - val_accuracy: 0.7638
Epoch 4/20
84/84 [==============================] - 1s 9ms/step - loss: 0.0744 - accuracy: 0.9764 - val_loss: 1.0140 - val_accuracy: 0.7017
Epoch 5/20
84/84 [==============================] - 1s 10ms/step - loss: 0.0439 - accuracy: 0.9882 - val_loss: 1.0642 - val_accuracy: 0.7043
Epoch 6/20
84/84 [==============================] - 1s 9ms/step - loss: 0.0825 - accuracy: 0.9779 - val_loss: 0.6947 - val_accuracy: 0.8180
Epoch 7/20
84/84 [==============================] - 1s 11ms/step - loss: 0.0988 - accuracy: 0.9698 - val_loss: 0.7306 - val_accuracy: 0.8381
Epoch 8/20
84/84 

In [6]:
#BUILDING INITIAL MODEL
new_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

#Compiling model
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
#TRAINING CNN
new_model.fit(new_train_images, new_train_labels, batch_size=55, epochs=20, validation_split = 0.2)

Epoch 1/20
336/336 [==============================] - 5s 11ms/step - loss: 3.4826 - accuracy: 0.7235 - val_loss: 1.2558 - val_accuracy: 0.5808
Epoch 2/20
336/336 [==============================] - 3s 8ms/step - loss: 0.3517 - accuracy: 0.8652 - val_loss: 1.0153 - val_accuracy: 0.6698
Epoch 3/20
336/336 [==============================] - 3s 8ms/step - loss: 0.2127 - accuracy: 0.9225 - val_loss: 1.0693 - val_accuracy: 0.6890
Epoch 4/20
336/336 [==============================] - 3s 8ms/step - loss: 0.1296 - accuracy: 0.9527 - val_loss: 1.2249 - val_accuracy: 0.7003
Epoch 5/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0882 - accuracy: 0.9712 - val_loss: 1.8788 - val_accuracy: 0.6176
Epoch 6/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0630 - accuracy: 0.9793 - val_loss: 2.0209 - val_accuracy: 0.6589
Epoch 7/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0434 - accuracy: 0.9864 - val_loss: 1.7669 - val_accuracy: 0.7031
Epoch

Then as a method for labelling the angles of the test data, we will first train a CNN model on the data from section 6, where each image is sorted into a folder by angle. We then use this trained angle model to predict the angle of each test image, and then sort the test images into the 3 angles using this prediction.

In [8]:
#BUILDING INITIAL MODEL
angle_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

#Compiling model
angle_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
#TRAINING CNN
angle_model.fit(angle_train_images, angle_train_labels, batch_size=55, epochs=20, validation_split = 0.2)

Epoch 1/20
267/267 [==============================] - 4s 11ms/step - loss: 7.6939 - accuracy: 0.8448 - val_loss: 0.3125 - val_accuracy: 0.8812
Epoch 2/20
267/267 [==============================] - 2s 9ms/step - loss: 0.2213 - accuracy: 0.9210 - val_loss: 0.2387 - val_accuracy: 0.9185
Epoch 3/20
267/267 [==============================] - 2s 9ms/step - loss: 0.1552 - accuracy: 0.9444 - val_loss: 0.2717 - val_accuracy: 0.9191
Epoch 4/20
267/267 [==============================] - 2s 9ms/step - loss: 0.1129 - accuracy: 0.9641 - val_loss: 0.4113 - val_accuracy: 0.8994
Epoch 5/20
267/267 [==============================] - 2s 9ms/step - loss: 0.0817 - accuracy: 0.9753 - val_loss: 0.2454 - val_accuracy: 0.9365
Epoch 6/20
267/267 [==============================] - 2s 8ms/step - loss: 0.0606 - accuracy: 0.9817 - val_loss: 0.4317 - val_accuracy: 0.9158
Epoch 7/20
267/267 [==============================] - 2s 8ms/step - loss: 0.0621 - accuracy: 0.9858 - val_loss: 0.3663 - val_accuracy: 0.9101
Epoch

In [10]:
test_angle_predictions = angle_model.predict(test_images)
test_angle_pred_labels = np.argmax(test_angle_predictions, axis =1)

41/41 [==============================] - 0s 3ms/step


In [11]:
test_sideon_images=[]
test_sideon_labels=[]

test_topdown_images=[]
test_topdown_labels=[]

test_back_images=[]
test_back_labels=[]

for i in range(len(test_images)):
    if (test_angle_pred_labels[i]==0):
        test_back_images.append(test_images[i])
        test_back_labels.append(test_labels[i])
    if (test_angle_pred_labels[i]==1):
        test_sideon_images.append(test_images[i])
        test_sideon_labels.append(test_labels[i])
    if (test_angle_pred_labels[i]==2):
        test_topdown_images.append(test_images[i])
        test_topdown_labels.append(test_labels[i])
        

Finally we are in a position to evaluate the performance of each of these models. First we calculate the accuracy of both 'model' and 'new_model' on the entire test data. Here we can see that the accuracy is improved by the new_model, suggesting that augmenting the images as in the preprocessing stage helped the model train better in differentiating between the image classes.

In [12]:
#CALCULATING TEST PREDICTION
test_loss = model.evaluate(test_images, test_labels)
new_test_loss = new_model.evaluate(test_images, test_labels)

41/41 [==============================] - 0s 3ms/step - loss: 0.4309 - accuracy: 0.9458


Next we evaluate the new model on the test data separated by angle. As we can see, the predictive accuracy given below seems to give evidence to our hypothesis that there is a great disparity between the performance of the model on 'Top-Down' images relative to any other image.

Whilst the lack of interpretability in the CNN makes it difficult to pinpoint exactly why this may be, an intuitive suggestion is the same one as in section 4: that the 'Top-Down' images contain less artifacts within the images. For instance, most of the top down images, especially in the 'notumour' folder, are simply a relatively undetailed grey circle for the brain with a very identifiable white circle surrounding it. However for the 'Side-On' and 'Back' images, not only is the overall shape of the head inconsistent, we now have to deal with other objects such as noses, eye sockets, jawbones and teeth etc. 

In [13]:
sideon_test_loss = new_model.evaluate(np.array(test_sideon_images), np.array(test_sideon_labels))
topdown_test_loss = new_model.evaluate(np.array(test_topdown_images), np.array(test_topdown_labels))
back_test_loss = new_model.evaluate(np.array(test_back_images), np.array(test_back_labels))

8/8 [==============================] - 0s 4ms/step - loss: 0.5912 - accuracy: 0.9098


To conclude then, we can have seen both the image augmentation modification and the identifying of image angle both have a positive effect on the performance of our neural network. 

As an extension, we could look at how to best mitigate the problem of differences in performance between angles. So far we have shown the problem exists, and we can give different levels of uncertainty to a prediction based on its angle, but no actual solution of how to improve the model. To do this we suggest predicting the angle of every image in the training set using the angle CNN (angle_model), and somehow incorporating these new labels into the CNN to retrain the model on both the images and angle labels simultaneously.